In [1]:
!pip install opencv-contrib-python==3.4.0.12 # otherwise you'd get an error message saying this when you try to use SIFT
                                   # This algorithm is patented and is excluded in this configuration; 
                                   # Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in 
                                   # function 'cv::xfeatures2d::SIFT::create'

     |████████████████████████████████| 30.5MB 163kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [2]:
import numpy as np
from scipy import ndimage, signal
from scipy.ndimage import gaussian_filter
from copy import deepcopy
import matplotlib.pyplot as plt

import random
import math
import os

!pip install opencv-python
import cv2

import plotly.graph_objects as go

from google.colab.patches import cv2_imshow

In [3]:
GAUSSIAN_BLUR_SIGMA=1

In [4]:
def compute_gaussian_subsample(image:np.ndarray, factor:int=2) -> np.ndarray:
    temp_image = gaussian_filter(deepcopy(image), sigma=GAUSSIAN_BLUR_SIGMA)

    len_x = temp_image.shape[0]
    len_y = temp_image.shape[1]
    half_len_x = int(len_x/2)
    half_len_y = int(len_y/2)

    # Subsample every other pixel
    temp_image = deepcopy(temp_image[::factor,::factor])

    # Gaussain blur
    temp_image = gaussian_filter(temp_image, sigma=GAUSSIAN_BLUR_SIGMA)

    return temp_image

In [5]:
def get_images(path0, path1, factor=2):
  img0 = cv2.imread(path0, cv2.IMREAD_GRAYSCALE)
  img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)

  # subsample because current imgs too big
  img0 = compute_gaussian_subsample(img0,factor)
  img1 = compute_gaussian_subsample(img1,factor)

  #plt.subplot(1, 2, 1)
  #plt.imshow(img0, cmap='gray')
  #plt.subplot(1, 2, 2)
  #plt.imshow(img1, cmap='gray')

  return img0, img1

In [6]:
# def perform_SIFT(img1, img2):
#   # temp naming everything SIFT
#   # sift = cv2.xfeatures2d.SIFT_create()
#   # kp1_SIFT, desc1_SIFT = sift.detectAndCompute(img1, None)
#   # kp2_SIFT, desc2_SIFT = sift.detectAndCompute(img2, None)

#   surf = cv2.xfeatures2d.SURF_create()
#   kp1_SIFT, desc1_SIFT = surf.detectAndCompute(img1, None)
#   kp2_SIFT, desc2_SIFT = surf.detectAndCompute(img2, None)

#   # orb = cv2.ORB_create(nfeatures=1000)
#   # kp1_SIFT, desc1_SIFT = orb.detectAndCompute(img1, None)
#   # kp2_SIFT, desc2_SIFT = orb.detectAndCompute(img2, None)

#   img1_SIFT = cv2.drawKeypoints(img1, kp1_SIFT, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS, color=(255,255,0))
#   img2_SIFT = cv2.drawKeypoints(img2, kp1_SIFT, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS, color=(255,255,0))

#   # #plt.figure()
#   # fig, axs = plt.subplots(2,figsize=(10,10))
#   # axs[0].imshow(img1_SIFT)
#   # #plt.subplot(1, 2, 2)
#   # axs[1].imshow(img2_SIFT)
#   # plt.show()

#   kp1 = kp1_SIFT
#   kp2 = kp2_SIFT
#   desc1 = desc1_SIFT
#   desc2 = desc2_SIFT

#   bf = cv2.BFMatcher()
#   matches = bf.knnMatch(desc1, desc2, k=2) # k=2 means find the top two matchs for each query descriptor

#   # Apply ratio test (as per David Lowe's SIFT paper: compare the best match with the 2nd best match_
#   good_matches = []
#   good_matches_without_list = []
#   for m,n in matches:
#       if m.distance < 0.75*n.distance: # only accept matchs that are considerably better than the 2nd best match
#           good_matches.append([m])
#           good_matches_without_list.append(m) # this is to simplify finding a homography later

#   # plt.figure()
#   # img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good_matches,
#   #                         None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS, 
#   #                         matchColor=(0,255,0))
#   # plt.figure(figsize = (10,10))
#   # plt.imshow(img3), plt.show()

#   src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)
#   dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)

#   return src_pts, dst_pts

In [7]:
# Modified SIFT code from assignment
def match_points(img1, img2, match_method='sift', ratio_param=0.75):
  
  if match_method == 'orb':
    orb = cv2.ORB_create(nfeatures=1000)
    kp1, desc1 = orb.detectAndCompute(img1, None)
    kp2, desc2 = orb.detectAndCompute(img2, None)
  elif match_method == 'surf':
    surf = cv2.xfeatures2d.SURF_create()
    kp1, desc1 = surf.detectAndCompute(img1, None)
    kp2, desc2 = surf.detectAndCompute(img2, None)
  else:
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)

  bf = cv2.BFMatcher()
  matches = bf.knnMatch(desc1, desc2, k=2) # k=2 means find the top two matchs for each query descriptor

  # Apply ratio test (as per David Lowe's SIFT paper: compare the best match with the 2nd best match_
  good_matches = []
  good_matches_without_list = []
  for m,n in matches:
      if m.distance < ratio_param*n.distance: # only accept matchs that are considerably better than the 2nd best match
          good_matches.append([m])
          good_matches_without_list.append(m) # this is to simplify finding a homography later

  src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)
  dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)

  return src_pts, dst_pts

In [8]:
def extract_calibration_vals(file):
  calib_vals = {}
  with open(file) as f:
    lines = f.readlines() # list containing lines of file
    for line in lines:
      line_split = line.split('=')
      if line_split[0] == 'cam0' or line_split[0] == 'cam1':
        matrix = line_split[1].strip('[]')
        matrix = matrix.split(';')
        matrix_vals = [[float(val.strip('[]')) for val in matrix_line.split()] for matrix_line in matrix]
        calib_vals[line_split[0]] = np.array(matrix_vals)
      else:
        calib_vals[line_split[0]] = float(line_split[1].rstrip())
  return calib_vals

#extract_calibration_vals('calib.txt')

In [9]:
# remove points that are n stdev away from the mean
def remove_outliers(world_coord, num_std=3):
  mean_coord = np.array([np.mean(world_coord[:,0]), np.mean(world_coord[:,1]), np.mean(world_coord[:,2])])
  stdev_coord = np.array([np.std(world_coord[:,0]), np.std(world_coord[:,1]), np.std(world_coord[:,2])])

  world_coord = world_coord[(world_coord[:,0] > mean_coord[0] - num_std*stdev_coord[0]) & (world_coord[:,0] < mean_coord[0] + num_std*stdev_coord[0])]
  world_coord = world_coord[(world_coord[:,1] > mean_coord[1] - num_std*stdev_coord[1]) & (world_coord[:,1] < mean_coord[1] + num_std*stdev_coord[1])]
  world_coord = world_coord[(world_coord[:,2] > mean_coord[2] - num_std*stdev_coord[2]) & (world_coord[:,2] < mean_coord[2] + num_std*stdev_coord[2])]

  return world_coord

In [10]:
# Get the 3D coordinates of the matches
def get_world_coord(path0, path1, folder, match_method='sift', ratio_param=0.75):
  factor = 2
  path0 = os.path.join(folder, path0)
  path1 = os.path.join(folder, path1)
  img0, img1 = get_images(path0, path1, factor)
  rgb0 = cv2.imread(path0)
  # Extract matching points from the imgs
  img0_pts, img1_pts = match_points(img0, img1, match_method, ratio_param)

  # Acquire camera information
  calib_vals = extract_calibration_vals(os.path.join(folder,'calib.txt'))
  K0 = calib_vals['cam0']
  K1 = calib_vals['cam1']
  f = K0[0,0]
  T = calib_vals['doffs']
  px0 = K0[0,2]
  px1 = K1[0,2]
  py0 = K0[1,2]
  py1 = K1[1,2]

  img0_pts = np.array(img0_pts)
  img1_pts = np.array(img1_pts)

  img0_pts = img0_pts.reshape((img0_pts.shape[0],img0_pts.shape[2]))
  img1_pts = img1_pts.reshape((img1_pts.shape[0],img1_pts.shape[2]))

  # Calculate world coordinates per pixel
  num_pts = len(img0_pts)
  world_coord = np.zeros((num_pts,6))
  for idx, pt in enumerate(img0_pts):
    # Calculate depth using: z = fT/(xr-xl)
    z = f*T/(np.absolute((pt[0])-(img1_pts[idx,0]))*factor)
    world_coord[idx,2] = z

    # x = fX/z + px, solve for X where X is world coordinate of x in the img
    # X = (x-px)z/f
    x0 = (pt[0]-px0)*factor*z/f
    x1 = (img1_pts[idx,0]-px1)*factor*z/f
    world_coord[idx,0] = -x0
    #world_coord[idx,0] = -(x0+x1)/2
    # do same for Y
    y0 = (pt[1]-py0)*factor*z/f
    y1 = (img1_pts[idx,1]-py1)*factor*z/f
    world_coord[idx,1] = y0
    #world_coord[idx,1] = (y0+y1)/2

    # add colour
    world_coord[idx,3:] = rgb0[int(pt[1])*factor,int(pt[0])*factor,:]

  return world_coord

# plot 3d point cloud of matches via plotly
def plot_pointCloud(pc, path='plot.html'):
    '''
    plots the Nx6 point cloud pc in 3D
    assumes (1,0,0), (0,1,0), (0,0,-1) as basis
    '''
    fig = go.Figure(data=[go.Scatter3d(
        x=pc[:, 0],
        y=pc[:, 1],
        z=-pc[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=pc[:, 3:][..., ::-1],
            opacity=0.8
        )
    )])
    fig.write_html(path)
    #fig.show()



In [11]:
def test_imgs(path0, path1, folder='', match_method='sift'):
  factor = 2
  path0 = os.path.join(folder, path0)
  path1 = os.path.join(folder, path1)
  img0, img1 = get_images(path0, path1, factor)
  rgb0 = cv2.imread(path0)

  # Extract matching points from the imgs
  img0_pts, img1_pts = match_points(img0, img1, match_method)

  img0_pts = np.array(img0_pts)
  img1_pts = np.array(img1_pts)
  img0_pts = img0_pts.reshape((img0_pts.shape[0],img0_pts.shape[2]))
  img1_pts = img1_pts.reshape((img1_pts.shape[0],img1_pts.shape[2]))

  num_pts = len(img0_pts)
  test_coord = np.zeros((num_pts,6))
  for idx,pt in enumerate(img0_pts):
    test_coord[idx,0] = -pt[0]*factor
    test_coord[idx,1] = pt[1]*factor
    test_coord[idx,2] = 5
    test_coord[idx,3:] = rgb0[int(pt[1])*factor,int(pt[0])*factor,:]

  plot_pointCloud(test_coord)


In [12]:
def run(img0='im0.png', img1='im1.png', folder='', match_method='sift', ratio=0.75, outlier_bound=2, include_2d=True):
  if include_2d:
    # if you want to see the 2d point matches
    test_imgs(img0,img1,folder,match_method)
  world_coord = get_world_coord(img0, img1, folder, match_method, ratio)
  world_coord = remove_outliers(world_coord, outlier_bound)
  plot_pointCloud(world_coord)


In [13]:
!unzip Imgs.zip

Archive:  Imgs.zip
   creating: Imgs/Bicycle/
  inflating: Imgs/Bicycle/calib.txt  
  inflating: Imgs/Bicycle/im0.png    
  inflating: Imgs/Bicycle/im1.png    
   creating: Imgs/Jadeplant/
  inflating: Imgs/Jadeplant/calib.txt  
  inflating: Imgs/Jadeplant/im0.png  
  inflating: Imgs/Jadeplant/im1.png  
   creating: Imgs/Mask/
  inflating: Imgs/Mask/calib.txt     
  inflating: Imgs/Mask/im0.png       
  inflating: Imgs/Mask/im1.png       
   creating: Imgs/Recycling/
  inflating: Imgs/Recycling/calib.txt  
  inflating: Imgs/Recycling/im0.png  
  inflating: Imgs/Recycling/im1.png  
   creating: Imgs/Storage/
  inflating: Imgs/Storage/calib.txt  
  inflating: Imgs/Storage/im0.png    
  inflating: Imgs/Storage/im1.png    


In [14]:
def run_folder(pic):
  pic_list = ['Bicycle', 'JadePlant', 'Mask', 'Recycling', 'Storage']
  folder_name = os.path.join('Imgs', pic_list[pic])

  run('im0.png', 'im1.png', folder=folder_name, match_method = 'surf', ratio=0.75, outlier_bound=2, include_2d=False)

run_folder(3)